# Ejercicio 13: LangChain

LangChain es un _framework_ de código abierto diseñado para facilitar el desarrollo de aplicaciones que combinan modelos de lenguaje LLMs con datos, herramientas externas y memoria. Está especialmente pensado para construir aplicaciones complejas basadas en IA, como sistemas _Retrieval-Augmented Generation_, asistentes conversacionales inteligentes, agentes autónomos y sistemas con razonamiento compuesto.

## Parte 1: Carga y preprocesamiento del corpus

In [2]:
import os
import json
from kaggle.api.kaggle_api_extended import KaggleApi

# Leer el archivo desde ../api/kaggle.json
with open("../api/kaggle.json", "r") as f:
    creds = json.load(f)

# Crear la carpeta ~/.kaggle si no existe
kaggle_dir = os.path.join(os.path.expanduser("~"), ".kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Escribir kaggle.json en la ubicación correcta
kaggle_json_path = os.path.join(kaggle_dir, "kaggle.json")
with open(kaggle_json_path, "w") as f:
    json.dump(creds, f)

# Cambiar permisos (solo lectura para el usuario)
os.chmod(kaggle_json_path, 0o600)

# Autenticar con Kaggle
api = KaggleApi()
api.authenticate()

In [20]:
import os
from dotenv import load_dotenv
import kaggle
import pandas as pd

# Cargar variables de entorno desde .env
load_dotenv(dotenv_path="../api/GOOGLE_API_KEY.env")

# Obtener la clave
openai_api_key = os.getenv("GOOGLE_API_KEY")

# Descargar el dataset (si no existe)
dataset = "rajneesh231/lex-fridman-podcast-transcript"
path = "../data/13langchain"

kaggle.api.dataset_download_files(dataset, path=path, unzip=True)

Dataset URL: https://www.kaggle.com/datasets/rajneesh231/lex-fridman-podcast-transcript


In [4]:
# Cargar el dataset CSV
file_path = os.path.join(path, "podcastdata_dataset.csv")  # ajusta nombre si difiere
df = pd.read_csv(file_path)
df

,id,guest,title,text
0,1,Max Tegmark,Life 3.0,"As part of MIT course 6S099, Artificial Genera..."
1,2,Christof Koch,Consciousness,As part of MIT course 6S099 on artificial gene...
2,3,Steven Pinker,AI in the Age of Reason,"You've studied the human mind, cognition, lang..."
3,4,Yoshua Bengio,Deep Learning,What difference between biological neural netw...
4,5,Vladimir Vapnik,Statistical Learning,The following is a conversation with Vladimir ...
...,...,...,...,...
314,321,Ray Kurzweil,"Singularity, Superintelligence, and Immortality","By the time he gets to 2045, we'll be able to ..."
315,322,Rana el Kaliouby,"Emotion AI, Social Robots, and Self-Driving Cars","there's a broader question here, right? As we ..."
316,323,Will Sasso,"Comedy, MADtv, AI, Friendship, Madness, and Pr...",Once this whole thing falls apart and we are c...
317,324,Daniel Negreanu,Poker,you could be the seventh best player in the wh...


In [6]:
pip install langchain


     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     ---------------------------------------- 0.0/43.1 kB ? eta -:--:--
     --------- ------------------------------ 10.2/43.1 kB ? eta -:--:--
     --------------------------- ---------- 30.7/43.1 kB 330.3 kB/s eta 0:00:01
     -------------------------------------- 43.1/43.1 kB 301.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.0 MB 825.8 kB/s eta 0:00:02
   --- ------------------------------------ 0.1/1.0 MB 751.6 kB/s eta 0:00:02
   ---- ----------------------------------- 0.1/1.0 MB 737.3 kB/s eta 0:00:02
   ----- ---------------------------------- 0.1/1.0 MB 711.9 kB/s eta 0:00:02
   ------ --------------------------------- 0.2/1.0 MB 655.4 kB/s eta 0:00:02
   ------- -------------------------------- 0.2/1.0 MB 619.5 kB/s eta 0:00:02
   -------- -----


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from langchain.schema import Document

# Convertir cada fila en un Document
documents = [
    Document(
        page_content=row["text"],
        metadata={"id": row["id"], "guest": row["guest"], "title": row["title"]},
    )
    for _, row in df.iterrows()
]

## Parte 2: Segmentación y embeddings

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

In [10]:
pip install -U langchain-community


     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ---------------------------------------- 0.0/76.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/76.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/76.3 kB ? eta -:--:--
     ----- ---------------------------------- 10.2/76.3 kB ? eta -:--:--
     --------------- ---------------------- 30.7/76.3 kB 146.3 kB/s eta 0:00:01
     --------------- ---------------------- 30.7/76.3 kB 146.3 kB/s eta 0:00:01
     --------------- ---------------------- 30.7/76.3 kB 146.3 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 122.9 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 122.9 kB/s eta 0:00:01
     -------------------- ----------------- 41.0/76.3 kB 122.9 kB/s eta 0:00:01
     ------------------------------ ------- 61.4/76.3 kB 121.3 kB/s eta 0:00:01
     -------------------------------------- 76.3/76.3 kB 140.9 kB/s eta 0:00:


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

texts = [chunk.page_content for chunk in chunks]
vectorstore = FAISS.from_texts(texts, embeddings)

vectorstore.save_local("index_13langchain")

C:\Users\ELI\AppData\Local\Temp\ipykernel_5296\2116709975.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


## Parte 3: Indexación en FAISS

In [12]:
vectorstore = FAISS.from_texts(texts, embeddings)
vectorstore.save_local("index_13langchain_02")

In [14]:
pip install google-generativeai

INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/155.4 kB ? eta -:--:--
   --------------- ------------------------ 61.4/155.4 kB 1.7 MB/s eta 0:00:01
   ----------------------- ---------------- 92.2/155.4 kB 1.1 MB/s eta 0:00:01
   ------------------------------------ --- 143.4/155.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 155.4/155.4 kB 1.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   - -------------------------------------- 0.1/1.3 MB 1.6 MB/s eta 0:00:01
   --- ------------------------------------ 0.1/1.3 MB 1.8 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.3 MB 1.7 MB/s eta 0:00:01
   ------- -------------------------------- 0.3/1.3 MB 1.4 MB/s eta 0:00:01
   --------- ------------------------------ 0.3/1.3 MB 1.5 MB/s eta 0:00:01
   ---------

  You can safely remove it manually.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
for m in genai.list_models():
    print(m.name, m.supported_generation_methods)

models/embedding-gecko-001 ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest ['generateContent', 'countTokens']
models/gemini-pro-vision ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest ['generateContent', 'countTokens']
models/gemini-1.5-pro-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest ['generateContent', 'countTokens']
models/gemini-1.5-flash ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-001 ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-1.5-flash-8b-latest ['createCachedContent', 'generateContent', 'countTokens']
models/gemini-2.5-pro-preview-03-25 ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/ge

## Parte 4: Creación de la cadena de recuperación

In [23]:
pip install langchain-google-genai


   ---------------------------------------- 0.0/47.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/47.8 kB ? eta -:--:--
   -------- ------------------------------- 10.2/47.8 kB ? eta -:--:--
   -------- ------------------------------- 10.2/47.8 kB ? eta -:--:--
   ------------------------- -------------- 30.7/47.8 kB 187.9 kB/s eta 0:00:01
   ---------------------------------- ----- 41.0/47.8 kB 164.3 kB/s eta 0:00:01
   ---------------------------------------- 47.8/47.8 kB 185.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI

# Inicializar Gemini LLM
llm = ChatGoogleGenerativeAI(model="models/gemini-1.5-flash", temperature=0)

# Crear retriever desde tu índice vectorial (FAISS o Chroma)
retriever = vectorstore.as_retriever()

# Cadena de pregunta-respuesta
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# Ejecutar una pregunta
response = qa_chain.run("What is AGI Artificial General Intelligence?")
print(response)

C:\Users\ELI\AppData\Local\Temp\ipykernel_5296\1339226018.py:14: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run("What is AGI Artificial General Intelligence?")


Based on the provided text, AGI, or Artificial General Intelligence, is a type of AI with the ability to learn and adapt to new environments.  However, it's unclear if a machine with AGI would necessarily have consciousness.  The text also notes that the term is widely used and understood, even though its precise definition is debated, with some considering it simply very smart AI surpassing human intelligence, while others equate it to human-like intelligence.


In [25]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
Usa solo el siguiente contexto para responder a la pregunta.
Si la respuesta no está explícita en el contexto, responde exactamente:
"No encontré información suficiente en el corpus."

Contexto:
{context}

Pregunta: {question}
Respuesta:
"""
)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})
print(result["result"])

No encontré información suficiente en el corpus.


In [26]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True
)

result = qa_chain.invoke({"query": "¿Qué es AGI Artificial General Intelligence?"})
print(result["result"])
print(result["source_documents"])

Based on the provided text, AGI (Artificial General Intelligence) is a term used to describe artificial intelligence that reaches or surpasses human-level intelligence.  However, the text also notes that the term is not well-defined and that human intelligence itself is not truly "general" but rather highly specialized.  The text mentions AIXI as an example of a theoretical system that could be considered truly general intelligence within the constraints of computational theory.  There's no single agreed-upon test to definitively determine if AGI has been achieved.
[Document(id='7bc570dd-374e-44a0-a010-daf927ae62fc', metadata={}, page_content="general intelligence, artificial intelligence, only refers to if you achieve human level or a subhuman level, but quite broad, is it also general intelligence? So we have to distinguish, or it's only super human intelligence, general artificial intelligence. Is there a test in your mind, like the Turing test for natural language or some other tes